In [253]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import keras
import warnings

from sklearn.preprocessing import MinMaxScaler
from hpelm import ELM
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
warnings.filterwarnings("ignore")

In [254]:
data = pd.read_csv("Group_14_data_cleaned.csv", index_col=0)
data.head()

,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
Auction_ID,,,,,,,,,,,
732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,0.444444,0
732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,0.444444,0
732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,0.444444,0
732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,0.444444,0
900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,0.666667,0


In [255]:
data['Class'].value_counts()

0    5646
1     675
Name: Class, dtype: int64

In [256]:
# Split the data into training and testing sets
X = data.drop(['Class', 'Bidder_ID'], axis=1)
y = data['Class']

## ELM

In [259]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert input data to numpy arrays
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

# print(np.unique(X_train))
# Initialize the ELM model with the desired parameters
model = ELM(X_train.shape[1], 1, classification="c", batch=10)

model.add_neurons(256, "tanh")


# Train the model
model.train(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1114   19]
 [   3  129]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1133
           1       0.87      0.98      0.92       132

    accuracy                           0.98      1265
   macro avg       0.93      0.98      0.96      1265
weighted avg       0.98      0.98      0.98      1265



In [221]:
class ELMClassifier:
    def __init__(self, n_hidden=10, activation="tanh", batch=10):
        self.n_hidden = n_hidden
        self.activation = activation
        self.batch = batch
        self.elm = ELM(inputs=X_train.shape[1], outputs=1, classification="c", batch=batch)
        self.elm.add_neurons(n_hidden, activation)

    def fit(self, X, y):
        self.elm.train(X, y)

    def predict(self, X):
        return self.elm.predict(X)

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

    def get_params(self, deep=True):
        return {'n_hidden': self.n_hidden, 'activation': self.activation, 'batch': self.batch}

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self


# Define the hyperparameters and their ranges
param_grid = {
    'n_hidden': [32, 64, 128, 256],
    'activation': ['tanh']
}

# elm_classifier = ELMClassifier()
# Initialize the random search object
elm_random = RandomizedSearchCV(estimator= ELMClassifier(), param_distributions=param_grid, n_iter=10, cv=5, random_state=42)

# Fit the random search object to the training data
random_search_result = elm_random.fit(X_train, y_train)

# Print the results
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))

# Print the best parameters and corresponding score
print("Best parameters: ", random_search_result.best_params_)
print("Best score: ", random_search_result.best_score_)


Best: nan using {'n_hidden': 32, 'activation': 'tanh'}
Best parameters:  {'n_hidden': 32, 'activation': 'tanh'}
Best score:  nan


## SVM with Linear Kernel 

In [33]:
from scipy import stats
# RANDOM SEARCH FOR 20 COMBINATIONS OF PARAMETERS
rand_list = {"C": stats.uniform(2, 10),
"gamma": stats.uniform(0.1, 5),
'kernel': ['linear']}
rand_search = RandomizedSearchCV(mdl, param_distributions = rand_list, n_iter = 50, n_jobs = 4, cv = 3, random_state = 2017, scoring = auc) 
rand_search.fit(X_train, y_train) 
# rand_search.cv_results_

# Print the best hyperparameters and the corresponding accuracy score
print("Best hyperparameters: ", rand_search.best_params_)
print("Accuracy score: ", rand_search.best_score_)


Best hyperparameters:  {'C': 2.209602254061174, 'gamma': 3.935350823412439, 'kernel': 'linear'}
Accuracy score:  0.9829980312538452


In [34]:
# Use the best hyperparameters to create a new SVM model
best_svm_model = SVC(C=rand_search.best_params_['C'], kernel=rand_search.best_params_['kernel'], gamma=rand_search.best_params_['gamma'])

# Train the new SVM model using the training data
best_svm_model.fit(X_train, y_train)

# Use the trained classifier to predict the class labels for the test data
y_pred = best_svm_model.predict(X_test)


# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1110   21]
 [   3  131]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1131
           1       0.86      0.98      0.92       134

    accuracy                           0.98      1265
   macro avg       0.93      0.98      0.95      1265
weighted avg       0.98      0.98      0.98      1265



## SVM with non linear kernel

In [31]:
from scipy import stats
# RANDOM SEARCH FOR 20 COMBINATIONS OF PARAMETERS
rand_list = {"C": stats.uniform(2, 10),
"gamma": stats.uniform(0.1, 1),
'kernel': ['poly', 'rbf']}
rand_search = RandomizedSearchCV(mdl, param_distributions = rand_list, n_iter = 50, n_jobs = 4, cv = 3, random_state = 2017, scoring = auc) 
rand_search.fit(X_train, y_train) 
# rand_search.cv_results_

# Print the best hyperparameters and the corresponding accuracy score
print("Best hyperparameters: ", rand_search.best_params_)
print("Accuracy score: ", rand_search.best_score_)


Best hyperparameters:  {'C': 9.087699346650997, 'gamma': 1.0163457603417565, 'kernel': 'rbf'}
Accuracy score:  0.9886417589006565


In [32]:
# Use the best hyperparameters to create a new SVM model
best_svm_model = SVC(C=rand_search.best_params_['C'], kernel=rand_search.best_params_['kernel'], gamma=rand_search.best_params_['gamma'])

# Train the new SVM model using the training data
best_svm_model.fit(X_train, y_train)

# Use the trained classifier to predict the class labels for the test data
y_pred = best_svm_model.predict(X_test)

# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1126    5]
 [   5  129]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1131
           1       0.96      0.96      0.96       134

    accuracy                           0.99      1265
   macro avg       0.98      0.98      0.98      1265
weighted avg       0.99      0.99      0.99      1265



## Training a Neural Network

In [267]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint


# Define the neural network model
def create_model(num_layers=1, num_neurons=10):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train.shape[1], activation='relu'))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier with fixed batch_size and epochs
model = KerasClassifier(build_fn=create_model, epochs=10, verbose=2)

# Define the hyperparameter distribution to search
param_dist = {
    'num_layers': [1,2,3],
    'num_neurons': [8,16,32,64,128],
}

# Perform randomized search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=3, n_iter=10)
random_search_result = random_search.fit(X_train, y_train)

# Print the results
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


Epoch 1/10
106/106 - 1s - loss: 0.5890 - accuracy: 0.8739 - 719ms/epoch - 7ms/step
Epoch 2/10
106/106 - 0s - loss: 0.2024 - accuracy: 0.9421 - 143ms/epoch - 1ms/step
Epoch 3/10
106/106 - 0s - loss: 0.0723 - accuracy: 0.9730 - 144ms/epoch - 1ms/step
Epoch 4/10
106/106 - 0s - loss: 0.0544 - accuracy: 0.9766 - 144ms/epoch - 1ms/step
Epoch 5/10
106/106 - 0s - loss: 0.0495 - accuracy: 0.9772 - 143ms/epoch - 1ms/step
Epoch 6/10
106/106 - 0s - loss: 0.0475 - accuracy: 0.9810 - 142ms/epoch - 1ms/step
Epoch 7/10
106/106 - 0s - loss: 0.0451 - accuracy: 0.9825 - 141ms/epoch - 1ms/step
Epoch 8/10
106/106 - 0s - loss: 0.0442 - accuracy: 0.9819 - 144ms/epoch - 1ms/step
Epoch 9/10
106/106 - 0s - loss: 0.0428 - accuracy: 0.9816 - 142ms/epoch - 1ms/step
Epoch 10/10
106/106 - 0s - loss: 0.0422 - accuracy: 0.9822 - 151ms/epoch - 1ms/step
53/53 - 0s - loss: 0.0511 - accuracy: 0.9786 - 244ms/epoch - 5ms/step
Epoch 1/10
106/106 - 1s - loss: 0.5657 - accuracy: 0.7532 - 834ms/epoch - 8ms/step
Epoch 2/10
106/1

106/106 - 0s - loss: 0.0478 - accuracy: 0.9801 - 183ms/epoch - 2ms/step
Epoch 3/10
106/106 - 0s - loss: 0.0421 - accuracy: 0.9840 - 160ms/epoch - 2ms/step
Epoch 4/10
106/106 - 0s - loss: 0.0385 - accuracy: 0.9840 - 145ms/epoch - 1ms/step
Epoch 5/10
106/106 - 0s - loss: 0.0361 - accuracy: 0.9834 - 142ms/epoch - 1ms/step
Epoch 6/10
106/106 - 0s - loss: 0.0348 - accuracy: 0.9858 - 150ms/epoch - 1ms/step
Epoch 7/10
106/106 - 0s - loss: 0.0338 - accuracy: 0.9864 - 177ms/epoch - 2ms/step
Epoch 8/10
106/106 - 0s - loss: 0.0336 - accuracy: 0.9846 - 180ms/epoch - 2ms/step
Epoch 9/10
106/106 - 0s - loss: 0.0320 - accuracy: 0.9861 - 183ms/epoch - 2ms/step
Epoch 10/10
106/106 - 0s - loss: 0.0298 - accuracy: 0.9902 - 181ms/epoch - 2ms/step
53/53 - 0s - loss: 0.0557 - accuracy: 0.9798 - 183ms/epoch - 3ms/step
Epoch 1/10
106/106 - 1s - loss: 0.1927 - accuracy: 0.9389 - 724ms/epoch - 7ms/step
Epoch 2/10
106/106 - 0s - loss: 0.0483 - accuracy: 0.9792 - 147ms/epoch - 1ms/step
Epoch 3/10
106/106 - 0s - l

Epoch 4/10
106/106 - 0s - loss: 0.0706 - accuracy: 0.9748 - 133ms/epoch - 1ms/step
Epoch 5/10
106/106 - 0s - loss: 0.0567 - accuracy: 0.9783 - 134ms/epoch - 1ms/step
Epoch 6/10
106/106 - 0s - loss: 0.0508 - accuracy: 0.9825 - 180ms/epoch - 2ms/step
Epoch 7/10
106/106 - 0s - loss: 0.0478 - accuracy: 0.9831 - 167ms/epoch - 2ms/step
Epoch 8/10
106/106 - 0s - loss: 0.0459 - accuracy: 0.9825 - 170ms/epoch - 2ms/step
Epoch 9/10
106/106 - 0s - loss: 0.0448 - accuracy: 0.9828 - 171ms/epoch - 2ms/step
Epoch 10/10
106/106 - 0s - loss: 0.0432 - accuracy: 0.9837 - 142ms/epoch - 1ms/step
53/53 - 0s - loss: 0.0486 - accuracy: 0.9781 - 202ms/epoch - 4ms/step
Epoch 1/10
106/106 - 1s - loss: 0.6478 - accuracy: 0.7093 - 735ms/epoch - 7ms/step
Epoch 2/10
106/106 - 0s - loss: 0.4922 - accuracy: 0.9733 - 144ms/epoch - 1ms/step
Epoch 3/10
106/106 - 0s - loss: 0.2813 - accuracy: 0.9775 - 166ms/epoch - 2ms/step
Epoch 4/10
106/106 - 0s - loss: 0.1146 - accuracy: 0.9780 - 172ms/epoch - 2ms/step
Epoch 5/10
106/1

106/106 - 0s - loss: 0.0301 - accuracy: 0.9869 - 197ms/epoch - 2ms/step
Epoch 6/10
106/106 - 0s - loss: 0.0306 - accuracy: 0.9869 - 193ms/epoch - 2ms/step
Epoch 7/10
106/106 - 0s - loss: 0.0294 - accuracy: 0.9887 - 194ms/epoch - 2ms/step
Epoch 8/10
106/106 - 0s - loss: 0.0257 - accuracy: 0.9881 - 177ms/epoch - 2ms/step
Epoch 9/10
106/106 - 0s - loss: 0.0241 - accuracy: 0.9899 - 162ms/epoch - 2ms/step
Epoch 10/10
106/106 - 0s - loss: 0.0220 - accuracy: 0.9905 - 164ms/epoch - 2ms/step
53/53 - 0s - loss: 0.0579 - accuracy: 0.9822 - 254ms/epoch - 5ms/step
Epoch 1/10
106/106 - 1s - loss: 0.1681 - accuracy: 0.9419 - 875ms/epoch - 8ms/step
Epoch 2/10
106/106 - 0s - loss: 0.0435 - accuracy: 0.9795 - 183ms/epoch - 2ms/step
Epoch 3/10
106/106 - 0s - loss: 0.0401 - accuracy: 0.9813 - 177ms/epoch - 2ms/step
Epoch 4/10
106/106 - 0s - loss: 0.0374 - accuracy: 0.9840 - 190ms/epoch - 2ms/step
Epoch 5/10
106/106 - 0s - loss: 0.0333 - accuracy: 0.9864 - 154ms/epoch - 1ms/step
Epoch 6/10
106/106 - 0s - l

In [ ]:
# Best: 0.982199 using {'num_neurons': 64, 'num_layers': 2}

## Training the best architecture

In [268]:

best_num_layers = random_search_result.best_params_["num_layers"]
best_num_neurons = random_search_result.best_params_["num_neurons"]
model = create_model(num_layers=best_num_layers, num_neurons=best_num_neurons)
model.fit(X_train, y_train, epochs=50, verbose=2)


Epoch 1/50
158/158 - 1s - loss: 0.0918 - accuracy: 0.9686 - 758ms/epoch - 5ms/step
Epoch 2/50
158/158 - 0s - loss: 0.0420 - accuracy: 0.9828 - 237ms/epoch - 2ms/step
Epoch 3/50
158/158 - 0s - loss: 0.0371 - accuracy: 0.9846 - 249ms/epoch - 2ms/step
Epoch 4/50
158/158 - 0s - loss: 0.0346 - accuracy: 0.9842 - 233ms/epoch - 1ms/step
Epoch 5/50
158/158 - 0s - loss: 0.0293 - accuracy: 0.9877 - 231ms/epoch - 1ms/step
Epoch 6/50
158/158 - 0s - loss: 0.0251 - accuracy: 0.9889 - 233ms/epoch - 1ms/step
Epoch 7/50
158/158 - 0s - loss: 0.0187 - accuracy: 0.9921 - 235ms/epoch - 1ms/step
Epoch 8/50
158/158 - 0s - loss: 0.0103 - accuracy: 0.9966 - 234ms/epoch - 1ms/step
Epoch 9/50
158/158 - 0s - loss: 0.0100 - accuracy: 0.9976 - 281ms/epoch - 2ms/step
Epoch 10/50
158/158 - 0s - loss: 0.0049 - accuracy: 0.9984 - 313ms/epoch - 2ms/step
Epoch 11/50
158/158 - 0s - loss: 0.0043 - accuracy: 0.9988 - 244ms/epoch - 2ms/step
Epoch 12/50
158/158 - 0s - loss: 0.0040 - accuracy: 0.9988 - 234ms/epoch - 1ms/step
E

In [269]:

# Use the trained classifier to predict the class labels for the test data
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

40/40 [==============================] - 0s 2ms/step
[[1128    5]
 [   1  131]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1133
           1       0.96      0.99      0.98       132

    accuracy                           1.00      1265
   macro avg       0.98      0.99      0.99      1265
weighted avg       1.00      1.00      1.00      1265



Both SVM and Neural Network performs with a similar accuaracy. Neural network performs slightly better than non-linear SVM. 

In [270]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)


# Print the accuracy without rounding
print("Accuracy: {:.4f}".format(accuracy))

40/40 [==============================] - 0s 2ms/step - loss: 0.0211 - accuracy: 0.9953
Accuracy: 0.9953


# Variable Selection

In [266]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


# Standardize the features using the StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit a Lasso regression model on the training data
alpha = 0.004  # regularization strength
lasso = Lasso(alpha=alpha)
lasso.fit(X_train, y_train)

# Select the features with non-zero coefficients
selected_features = X.columns[lasso.coef_ != 0]

# Print the selected features
print("Selected features:", selected_features)


Selected features: Index(['Successive_Outbidding', 'Last_Bidding', 'Winning_Ratio',
       'Auction_Duration'],
      dtype='object')


In [113]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


# Get the feature matrix X and target vector y
X_train_ = data.drop(['Bidder_ID', 'Class'], axis=1)
y_train_ = data['Class']

k = 4
knn = KNeighborsClassifier(n_neighbors=k) 
selector = SelectKBest(score_func=f_classif, k=k)
# selector = SelectKBest(score_func=chi2, k=k)
X_train_new = selector.fit_transform(X_train_, y_train_)


# Print the selected feature names
selected_features = X_train_.columns[selector.get_support()]
print("Selected features:", list(selected_features))



Selected features: ['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Winning_Ratio']


Training the model with the selected features to find which feature selection is a better technique. 

In [133]:
# Split the data into training and testing sets
X = data[['Bidder_Tendency', 'Successive_Outbidding', 'Last_Bidding',
       'Winning_Ratio']]
y = data['Class']
X_train_sl_ft_1, X_test_sl_ft_1, y_train_sl_ft_1, y_test_sl_ft_1 = train_test_split(X, y, test_size=0.2)

In [134]:
# Define the neural network model
def create_model_(num_layers=1, num_neurons=10):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train_.shape[1], activation='relu'))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [135]:
model = create_model_(num_layers=2, num_neurons=64)
model.fit(X_train_sl_ft_1, y_train_sl_ft_1, epochs=50, verbose=2)

Epoch 1/50
158/158 - 1s - loss: 0.2030 - accuracy: 0.9389 - 646ms/epoch - 4ms/step
Epoch 2/50
158/158 - 0s - loss: 0.0479 - accuracy: 0.9777 - 202ms/epoch - 1ms/step
Epoch 3/50
158/158 - 0s - loss: 0.0476 - accuracy: 0.9775 - 205ms/epoch - 1ms/step
Epoch 4/50
158/158 - 0s - loss: 0.0455 - accuracy: 0.9784 - 204ms/epoch - 1ms/step
Epoch 5/50
158/158 - 0s - loss: 0.0449 - accuracy: 0.9778 - 209ms/epoch - 1ms/step
Epoch 6/50
158/158 - 0s - loss: 0.0448 - accuracy: 0.9798 - 237ms/epoch - 1ms/step
Epoch 7/50
158/158 - 0s - loss: 0.0444 - accuracy: 0.9788 - 211ms/epoch - 1ms/step
Epoch 8/50
158/158 - 0s - loss: 0.0437 - accuracy: 0.9794 - 206ms/epoch - 1ms/step
Epoch 9/50
158/158 - 0s - loss: 0.0416 - accuracy: 0.9800 - 206ms/epoch - 1ms/step
Epoch 10/50
158/158 - 0s - loss: 0.0441 - accuracy: 0.9784 - 202ms/epoch - 1ms/step
Epoch 11/50
158/158 - 0s - loss: 0.0421 - accuracy: 0.9796 - 204ms/epoch - 1ms/step
Epoch 12/50
158/158 - 0s - loss: 0.0433 - accuracy: 0.9796 - 202ms/epoch - 1ms/step
E

In [132]:

# Use the trained classifier to predict the class labels for the test data
y_pred_ = model.predict(X_test_sl_ft_1)
y_pred_ = np.round(y_pred_sl_ft_1)
# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test_sl_ft_1, y_pred_sl_ft_1))
print(classification_report(y_test_sl_ft_1, y_pred_sl_ft_1))

40/40 [==============================] - 0s 2ms/step
[[1115   25]
 [   0  125]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1140
           1       0.83      1.00      0.91       125

    accuracy                           0.98      1265
   macro avg       0.92      0.99      0.95      1265
weighted avg       0.98      0.98      0.98      1265



In [250]:
#todo: train ELM on selected variables

In [137]:
# Split the data into training and testing sets
X = data[['Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Winning_Ratio']]
y = data['Class']
X_train_sl_ft_2, X_test_sl_ft_2, y_train_sl_ft_2, y_test_sl_ft_2 = train_test_split(X, y, test_size=0.2)
model = create_model_(num_layers=2, num_neurons=64)
model.fit(X_train_sl_ft_2, y_train_sl_ft_2, epochs=50, verbose=2)

# Use the trained classifier to predict the class labels for the test data
y_pred_ = model.predict(X_test_sl_ft_2)
y_pred_ = np.round(y_pred_sl_ft_2)
# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test_sl_ft_2, y_pred_sl_ft_2))
print(classification_report(y_test_sl_ft_2, y_pred_sl_ft_2))

Epoch 1/50
158/158 - 1s - loss: 0.2068 - accuracy: 0.9502 - 667ms/epoch - 4ms/step
Epoch 2/50
158/158 - 0s - loss: 0.0466 - accuracy: 0.9790 - 222ms/epoch - 1ms/step
Epoch 3/50
158/158 - 0s - loss: 0.0435 - accuracy: 0.9798 - 232ms/epoch - 1ms/step
Epoch 4/50
158/158 - 0s - loss: 0.0420 - accuracy: 0.9804 - 326ms/epoch - 2ms/step
Epoch 5/50
158/158 - 0s - loss: 0.0421 - accuracy: 0.9810 - 276ms/epoch - 2ms/step
Epoch 6/50
158/158 - 0s - loss: 0.0407 - accuracy: 0.9814 - 286ms/epoch - 2ms/step
Epoch 7/50
158/158 - 0s - loss: 0.0412 - accuracy: 0.9814 - 275ms/epoch - 2ms/step
Epoch 8/50
158/158 - 0s - loss: 0.0408 - accuracy: 0.9800 - 250ms/epoch - 2ms/step
Epoch 9/50
158/158 - 0s - loss: 0.0417 - accuracy: 0.9802 - 214ms/epoch - 1ms/step
Epoch 10/50
158/158 - 0s - loss: 0.0390 - accuracy: 0.9822 - 207ms/epoch - 1ms/step
Epoch 11/50
158/158 - 0s - loss: 0.0397 - accuracy: 0.9822 - 208ms/epoch - 1ms/step
Epoch 12/50
158/158 - 0s - loss: 0.0406 - accuracy: 0.9802 - 229ms/epoch - 1ms/step
E

NameError: name 'y_pred_sl_ft_2' is not defined

In [138]:
# Use the trained classifier to predict the class labels for the test data
y_pred_sl_ft_2 = model.predict(X_test_sl_ft_2)
y_pred_sl_ft_2 = np.round(y_pred_sl_ft_2)
# Evaluate the classifier's performance on the test data
print(confusion_matrix(y_test_sl_ft_2, y_pred_sl_ft_2))
print(classification_report(y_test_sl_ft_2, y_pred_sl_ft_2))

40/40 [==============================] - 0s 2ms/step
[[1102   25]
 [   0  138]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1127
           1       0.85      1.00      0.92       138

    accuracy                           0.98      1265
   macro avg       0.92      0.99      0.95      1265
weighted avg       0.98      0.98      0.98      1265



Both feature selections provide similar accuracies. 
But we are going to conisder Lasso feature selection method for our case, because Lasso is linear and uses L1 regularization to shrink coefficients of the features towards zero which can also prevent the problem of overfitting. 

## Using the best model from part 2 and part 3 on the selected features. 

In [139]:
#  Selected features : ['Bidder_Tendency', 'Successive_Outbidding', 'Last_Bidding', 'Winning_Ratio']

In [140]:
# Split the data into training and testing sets
X = data[['Bidder_Tendency', 'Successive_Outbidding', 'Last_Bidding',
       'Winning_Ratio']]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [147]:
d

0.9762845849802372


In [148]:
# Define the neural network model
def create_model_(num_layers=1, num_neurons=10):
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train_.shape[1], activation='relu'))
    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model_(num_layers=2, num_neurons=64)
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
158/158 - 1s - loss: 0.1913 - accuracy: 0.9409 - 766ms/epoch - 5ms/step
Epoch 2/50
158/158 - 0s - loss: 0.0458 - accuracy: 0.9794 - 231ms/epoch - 1ms/step
Epoch 3/50
158/158 - 0s - loss: 0.0430 - accuracy: 0.9816 - 340ms/epoch - 2ms/step
Epoch 4/50
158/158 - 0s - loss: 0.0425 - accuracy: 0.9814 - 279ms/epoch - 2ms/step
Epoch 5/50
158/158 - 0s - loss: 0.0409 - accuracy: 0.9812 - 259ms/epoch - 2ms/step
Epoch 6/50
158/158 - 0s - loss: 0.0409 - accuracy: 0.9814 - 236ms/epoch - 1ms/step
Epoch 7/50
158/158 - 0s - loss: 0.0430 - accuracy: 0.9808 - 210ms/epoch - 1ms/step
Epoch 8/50
158/158 - 0s - loss: 0.0410 - accuracy: 0.9808 - 223ms/epoch - 1ms/step
Epoch 9/50
158/158 - 0s - loss: 0.0408 - accuracy: 0.9808 - 244ms/epoch - 2ms/step
Epoch 10/50
158/158 - 0s - loss: 0.0409 - accuracy: 0.9816 - 283ms/epoch - 2ms/step
Epoch 11/50
158/158 - 0s - loss: 0.0403 - accuracy: 0.9814 - 294ms/epoch - 2ms/step
Epoch 12/50
158/158 - 0s - loss: 0.0410 - accuracy: 0.9800 - 227ms/epoch - 1ms/step
E

In [150]:
# Evaluate the model on the testing set
loss, accuracy_nn = model.evaluate(X_test, y_test)


40/40 [==============================] - 0s 2ms/step - loss: 0.0478 - accuracy: 0.9810


In [151]:
# Print the accuracy without rounding
print("Accuracy of the best part 2 model (Rf) on selected features : {:.4f}".format(acc_rf))
print("Accuracy of the best part 3 model (NN) on selected features: {:.4f}".format(accuracy_nn))

Accuracy of the best part 2 model (Rf) on selected features : 0.9763
Accuracy of the best part 3 model (NN) on selected features: 0.9810


In [224]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

rf = RandomForestClassifier(max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100)

sfs = SFS(rf,
          k_features=(1, X.shape[1]),
          forward=True,
          floating=True,
          scoring='accuracy',
          cv=5,
          n_jobs=-1)

sfs = sfs.fit(X_train, y_train)



# Get the selected column names
selected_columns = X.columns[list(sfs.k_feature_idx_)]

# Print the selected column names
print("Selected features (RF):", selected_columns)

# Print the best score
print("Best score (RF):", sfs.k_score_)

# Transform X to contain only the selected features
X_selected_rf = sfs.transform(X)

# Train RF and NN models with selected features
rf.fit(X_selected_rf, y)




Selected features (RF): Index(['Successive_Outbidding', 'Last_Bidding', 'Winning_Ratio',
       'Auction_Duration'],
      dtype='object')
Best score (RF): 0.9980229335022266


RandomForestClassifier(max_depth=20, min_samples_split=5)

In [249]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

X = X_train
y = y_train

# Scale data using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the neural network model
nn_model = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=100, solver='adam', 
                         verbose=0,  random_state=42, tol=0.0001)

# Define the Bi-directional Elimination wrapper method
sfs = SFS(nn_model, k_features=(1, X.shape[1]), forward=True, floating=True, verbose=2, scoring='accuracy', cv=5)

# Perform feature selection using the wrapper method
sfs = sfs.fit(X, y)

# Split the data into training and testing sets
X = data.drop(['Class', 'Bidder_ID'], axis=1)
# Get the selected column names
selected_columns = X.columns[list(sfs.k_feature_idx_)]

# Print the selected column names
print("Selected features:", selected_columns)

# Train neural network model with selected features
X_selected = sfs.transform(X_train)
nn_model.fit(X_selected, y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   31.3s finished

[2023-05-01 00:35:22] Features: 1/9 -- score: 0.9740917105515221[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   34.3s finished

[2023-05-01 00:35:57] Features: 2/9 -- score: 0.989716087464765[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   58.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=

Selected features: Index(['Bidder_Tendency', 'Successive_Outbidding', 'Early_Bidding',
       'Winning_Ratio', 'Auction_Duration'],
      dtype='object')


MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=100, random_state=42,
              verbose=0)

In [248]:
# Split the data into training and testing sets
X = data.drop(['Class', 'Bidder_ID'], axis=1)
y = data['Class']
selected_columns = X.columns[list(sfs.k_feature_idx_)]
print(selected_columns)

Index(['Bidder_Tendency', 'Successive_Outbidding', 'Early_Bidding',
       'Winning_Ratio', 'Auction_Duration'],
      dtype='object')


In [ ]:
# Print the accuracy without rounding
print("Accuracy of the best part 2 model (Rf) on selected features : {:.4f}".format(acc_rf))
print("Accuracy of the best part 3 model (NN) on selected features: {:.4f}".format(accuracy_nn))

In [251]:
# TODO: report test accuracies for both models on the given selected feature